In [1]:
!pip install pandasql
!pip install kaggle

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=bc5a7c88c857a9fa217d98e2628db96779d05c5c022b7f9defb7356aa62314d0
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [2]:
import pandas as pd
import numpy as np
import pandasql
import re
import nltk

In [3]:
nltk.download('universal_tagset')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Load raw dataset

Pulled and unzipped data from [kaggle](https://www.kaggle.com/kanaryayi/recipe-ingredients-and-reviews?select=reviews.csv) using Kaggle API. 

In [4]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/
!touch ~/.kaggle/kaggle.json

api_token = {"username":"songyuyve","key":"b1bb938576fca9501c01cca2c3b0fa22"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [131]:
!kaggle datasets download -d kanaryayi/recipe-ingredients-and-reviews -f recipes.csv
!kaggle datasets download -d kanaryayi/recipe-ingredients-and-reviews -f reviews.csv

401 - Unauthorized
401 - Unauthorized


In [132]:
!unzip recipes.csv.zip recipes.csv
!unzip reviews.csv.zip reviews.csv

Archive:  recipes.csv.zip
replace recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  reviews.csv.zip
replace reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# Import raw data

In [247]:
recipes_df = pd.read_csv('recipes.csv', sep=';',  encoding= 'unicode_escape' , quotechar="'")
reviews_df = pd.read_csv('reviews.csv', delimiter=';', encoding='unicode_escape' , quotechar="'")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [177]:
recipes_df.head(5)

,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions
0,Golden Crescent Rolls Recipe,7000,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ..."
1,Poppy Seed Bread with Glaze Recipe,7001,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....
2,Czech Christmas Hoska Recipe,7002,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 h,45 m,5 h 30 m,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...
3,Applesauce Bread I Recipe,7003,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....
4,Raisin Brown Bread Recipe,7004,5,https://images.media-allrecipes.com/userphotos...,Tina,30 m,40 m,3 h 10 m,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown..."


In [178]:
reviews_df.head(5)

,RecipeID,profileID,Rate,Comment
0,7000,675719,5,Oh Yay! I've never made any kind of bread fro...
1,7000,1478626,5,These rolls are wonderful. I had to bake them ...
2,7000,608663,5,This recipe was made for a group of 120 in a c...
3,7000,2785736,5,WOO HOO.. I did it with the help from my fello...
4,7000,594474,5,"These are great!! Not a croissant, but much li..."


# Prepare Recipe data

Below are steps we took to align data with our development needs:


1.   Sanity checking: 
  1. convert column with time, prefixed with time unitis, to numeric values with unit minutes.
  2. convert *RecipeID* to integers, remove rows with invalid records.
  3. drop recipes without *Directions*.
2.   Reformat *Ingredient* column, extract ingredient name, unit and quantity, and store this information within an independent table, referencing Recipe_df using *RecipeID*.
3.   Extract all unique ingredient items and create a Ingredient table, each ingredient identified by a primary key.



## Convert time string to numeric values in minutes

In [179]:
def time_to_min(row):
  ''' Unify time columns to minutes
  '''
  # check contains number
  if not any(char.isdigit() for char in row):
    return None
  
  day, hour, minut = 0, 0, 0
  hour_ind, day_ind = 0, 0
  if 'd' in row:
    day = float(row[:row.find('d')].strip(' '))
    row = row[row.find('d')+1:]
  if 'h' in row:
    hour = float(row[:row.find('h')].strip(' '))
    row = row[row.find('h')+1:]
  if 'm' in row:
    minut = float(row[:row.find('m')].strip(' '))
  return hour*60+minut+day*1440

In [180]:
recipes_df['Prepare_Time'] = recipes_df['Prepare Time'].apply(time_to_min)
recipes_df['Cook_Time'] = recipes_df['Cook Time'].apply(time_to_min)
recipes_df['Total_Time'] = recipes_df['Total Time'].apply(time_to_min)

recipes_df.drop(['Total Time', 'Cook Time', 'Prepare Time'], axis=1, inplace=True)
recipes_df.head(5)

,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
0,Golden Crescent Rolls Recipe,7000,304,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
1,Poppy Seed Bread with Glaze Recipe,7001,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
2,Czech Christmas Hoska Recipe,7002,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0
3,Applesauce Bread I Recipe,7003,124,https://images.media-allrecipes.com/userphotos...,GAF55,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....,10.0,80.0,90.0
4,Raisin Brown Bread Recipe,7004,5,https://images.media-allrecipes.com/userphotos...,Tina,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown...",30.0,40.0,190.0


## Convert *RecipeID* to Integer
To facilitate future table joins, convert *RecipeID* into integers. If a record is not a valid string-type integer, the entire row is removed.

In [213]:
def convert_float(row):
  row = str(row).strip()
  rate = 0
  try:
    rate = float(row)
  except ValueError:
    print(row)
  return rate

In [214]:
def convert_int(row):
  row = str(row).strip()
  id = 0
  try:
    id = int(row)
  except ValueError:
    print(row)
  return id

In [182]:
# Sanity check, if all rows contain invalid id
recipes_df[recipes_df['RecipeID'].apply(convert_int) == 0]

RecipeID


,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
245,Recipe Name,RecipeID,Review Count,Recipe Photo,Author,Ingredients,Directions,NaN,NaN,NaN


In [183]:
# Drop row 245
recipes_df.drop(index=245, inplace=True)

In [184]:
# Convert RecipeId to integers, Reset id to RecipeId
recipes_df['RecipeID'] = recipes_df['RecipeID'].apply(convert_int)
recipes_df.set_index(keys=['RecipeID'],inplace=True)
recipes_df.head(3)

,Recipe Name,Review Count,Recipe Photo,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
RecipeID,,,,,,,,,
7000,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
7001,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
7002,Czech Christmas Hoska Recipe,11,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0


## Recipe Missing Value Imputation and Removal
As the main purpose of our website is to guide users how to cook, we thus dropped recipes without ingredients or directions, retained recipes without cooking time estimation.

In [185]:
recipes_df.isna().sum()

Recipe Name        0
Review Count       0
Recipe Photo       0
Author             1
Ingredients      838
Directions       844
Prepare_Time    6518
Cook_Time       8486
Total_Time      6182
dtype: int64

In [186]:
# Remove recipes without ingredients or directions
recipes_df.dropna(axis=0, how='any', subset=['Ingredients', 'Directions'], inplace=True)
recipes_df.isna().sum()

Recipe Name        0
Review Count       0
Recipe Photo       0
Author             0
Ingredients        0
Directions         0
Prepare_Time    5675
Cook_Time       7643
Total_Time      5339
dtype: int64

In [187]:
recipes_df.drop(['Review Count'], axis=1, inplace=True)
recipes_df.rename({'Recipe Name':'RecipeName', 'Recipe Photo':'RecipePhoto'}, axis=1, inplace=True)
recipes_df

,RecipeName,RecipePhoto,Author,Ingredients,Directions,Prepare_Time,Cook_Time,Total_Time
RecipeID,,,,,,,,
7000,Golden Crescent Rolls Recipe,https://images.media-allrecipes.com/userphotos...,Mike A.,2 (.25 ounce) packages active dry yeast**3/4 c...,"Dissolve yeast in warm water.**Stir in sugar, ...",25.0,15.0,190.0
7001,Poppy Seed Bread with Glaze Recipe,https://images.media-allrecipes.com/userphotos...,Christina Jun,3 cups all-purpose flour**1 1/2 teaspoons salt...,Preheat oven to 350 degrees F (175 degrees C)....,15.0,60.0,80.0
7002,Czech Christmas Hoska Recipe,https://images.media-allrecipes.com/userphotos...,Carol Ponec-Nemec,1 (0.6 ounce) cake compressed fresh yeast**1/4...,Dissolve yeast in warm water.**In a large bowl...,60.0,45.0,330.0
7003,Applesauce Bread I Recipe,https://images.media-allrecipes.com/userphotos...,GAF55,3 cups all-purpose flour**3 eggs**2 cups white...,Preheat oven to 350 degrees F (175 degrees C)....,10.0,80.0,90.0
7004,Raisin Brown Bread Recipe,https://images.media-allrecipes.com/userphotos...,Tina,2 cups boiling water**1 cup rolled oats**2 tab...,"Combine boiling water, oats, shortening, brown...",30.0,40.0,190.0
...,...,...,...,...,...,...,...,...
27572,Johnsonville® Three Cheese Italian Style Chick...,https://images.media-allrecipes.com/images/795...,The Kitchen at Johnsonville Sausage,1 (12 inch) pre-baked pizza crust**1 1/2 cups ...,Top pizza crust with cheese. Bake crust accor...,NaN,NaN,30.0
27576,Johnsonville® Three Cheese Italian Style Chick...,https://images.media-allrecipes.com/images/795...,The Kitchen at Johnsonville Sausage,1 (12 inch) pre-baked pizza crust**1 1/2 cups ...,Top pizza crust with cheese. Bake crust accor...,NaN,NaN,30.0
27580,Johnsonville® Three Cheese Italian Style Chick...,https://images.media-allrecipes.com/images/795...,The Kitchen at Johnsonville Sausage,1 (12 inch) pre-baked pizza crust**1 1/2 cups ...,Top pizza crust with cheese. Bake crust accor...,NaN,NaN,30.0


# Extract Ingredients from Recipe

Now ingredients are stored within the *Ingredients* column in the format of {quantity}{unit}{item name}, for example, for recipe with ID 7000:
```
  recipes_df.iloc[0]['Ingredients'].split('**')
 ```
gives
```
 '2 (.25 ounce) packages active dry yeast',
 '3/4 cup warm water (110 degrees F/45 degrees C)',
 '1/2 cup white sugar',
 '1 teaspoon salt',
 '2 eggs',
 '1/2 cup butter, room temperature',
 '4 cups all-purpose flour',
 '1/4 cup butter, softened' 
 ```
Our goal is to extract quantity, unit, item name separately and attain an output table in following format:

|RecipeID| quantity | Unit | Ingredient|
| --- | --- | --- | ---|
|7000| 0.25 | ounce| yeast|
|7000|	1.5|	cup|	water|
|7000	|1.0	|cup	|sugar|
|7000	|1.0	|teaspoon	|salt|




In [188]:
Ingred = recipes_df['Ingredients']

In [189]:
# Unit set, only in Singular form
unit_set = ['cup','ounce','pound','teaspoon','quart','tablespoon','slice','package']
all_unit = "|".join(unit_set)

In [229]:
def extract_ingred(row):
  rows = row.split("**")
  res = []
  for row in rows:
    unit, quant, ing = None, 0, row

    # If contains () use units in ()
    if '(' in set(row) and row.find(')')!=len(row)-1:
      unit = row[row.find('(')+1:row.find(')')]
      quant = re.findall("\d+\.\d+|\d+|\d{2,}?",unit)
      if quant: 
        quant = float(quant[0])
      else:
        quant = 0
      measure = unit.split()[-1]
      unit = measure
      ing = (" ".join(row[row.find(')')+1:].split()[1:]))
      row = row[row.find(')')+1:]
    else:

      # Match quantity
      num = re.findall("\d+\/[1-9][0-9]*|\d+|\d{2,}|\d{3,}?",row)
      frac = 0
      # Handle fractions
      if num:
        if (len(num)>1 or "/" in num[0]):
        # Fraction
          for n in num:
            if "/" not in n:
              quant += int(n)
            else: 
              frac = n
              ind_slash = frac.find('/')
              frac = int(frac[ind_slash-1])/int(frac[ind_slash+1])
              quant+= frac
              break
        else:
          quant = float(num[0])
      
      # Store quantity as decimal
      quant = quant+frac

      # Match units
      unit = re.search(rf"{all_unit}",row)
      if unit:
        unit = unit.group()

      # Remove suffixes with , or ()
      if ',' in set(ing):
        ing = row[: row.find(',')]
      if '(' in set(ing):
        ing = row[: row.find('(')]
      ing = " ".join(re.findall(rf"(?!{all_unit})\b([a-z]+)?", ing)).strip()

      
    # Tokenize ingredients, store only nouns
    ing_set = nltk.word_tokenize(ing)
    tagged = nltk.pos_tag(ing_set, tagset = 'universal')
    ing = " ".join([word[0] for word in tagged if word[1]=='NOUN'])

    res.append((round(quant,2),unit,ing))
  return res

In [230]:
New_ingred = Ingred.apply(extract_ingred)

In [231]:
ingredient_df = pd.DataFrame(New_ingred.explode())
ingredient_df = ingredient_df.reset_index()

In [232]:
# Setup ingredient dataframe
ingredient_df[['quantity','unit', 'ingredient']] = pd.DataFrame(ingredient_df['Ingredients'].tolist())
ingredient_df.drop(['Ingredients'], axis=1, inplace=True)

In [233]:
ingredient_df.dropna(axis=0, subset=['ingredient'], inplace=True)
ingredient_df

,RecipeID,quantity,unit,ingredient
0,7000,25.0,ounce,yeast
1,7000,1.5,cup,water
2,7000,1.0,cup,sugar
3,7000,1.0,teaspoon,salt
4,7000,2.0,None,eggs
...,...,...,...,...
152858,27594,12.0,inch,pizza crust
152859,27594,2.0,cup,mozzarella cheese
152860,27594,14.0,ounce,pizza sauce
152861,27594,12.0,ounce,Johnsonville® Cheese Style Chicken Sausage


In [234]:
# Extract all unique ingredients, create an independent table with ingredient ID
unique_ingred = ingredient_df['ingredient'].unique()

In [235]:
unique_ingred_df = pd.DataFrame(unique_ingred, columns=['ingredient'])
unique_ingred_df.reset_index(inplace=True)
unique_ingred_df.rename({'index':'ingred_index'}, inplace=True, axis=1)
unique_ingred_df

In [237]:
# Add ingredientID to ingredient table
ingredient_df_indexed = ingredient_df.merge(unique_ingred_df, left_on='ingredient', right_on='ingredient', how='inner', copy=False)
ingredient_df_indexed

,RecipeID,quantity,unit,ingredient,ingred_index
0,7000,25.0,ounce,yeast,0
1,7002,0.6,ounce,yeast,0
2,7004,2.0,tablespoon,yeast,0
3,7008,2.0,tablespoon,yeast,0
4,7009,25.0,ounce,yeast,0
...,...,...,...,...,...
152858,27262,12.0,inch,pizza crust bread shell,3795
152859,27263,7.0,ounce,BUITONI® Pesto Sun-Dried Tomatoes,3796
152860,27263,0.5,cup,onion alfalfa sprouts,3797
152861,27277,3.0,tablespoon,sugar replacement,3798


# Clean reviews

In [248]:
reviews_df.isna().sum()

RecipeID       0
profileID    178
Rate         178
Comment      365
dtype: int64

In [249]:
# Drop rows without ratings, drop comment column
reviews_df.dropna(subset=['Rate'], axis=0, inplace=True)
reviews_df.drop(['Comment'], axis=1, inplace=True)

In [250]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1578722 entries, 0 to 1578899
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   RecipeID   1578722 non-null  object
 1   profileID  1578722 non-null  object
 2   Rate       1578722 non-null  object
dtypes: object(3)
memory usage: 48.2+ MB


In [251]:
reviews_df['RecipeID']= reviews_df['RecipeID'].apply(convert_int)
reviews_df['Rate'] = reviews_df['Rate'].apply(convert_float)

RecipeID
Rate


In [252]:
# Check if all recipesID in review exists in recipes
review_id = set(reviews_df.RecipeID.unique())
recipe_id = set(recipes_df.index.unique())
diff = list(review_id - recipe_id)

# Drop mismatch from review table
reviews_df.drop(index=(reviews_df[reviews_df['RecipeID'].isin(diff)]).index, inplace=True)
len(reviews_df)

1027292

In [244]:
reviews_df

,RecipeID,profileID,Rate
0,7000,675719,5.0
1,7000,1478626,5.0
2,7000,608663,5.0
3,7000,2785736,5.0
4,7000,594474,5.0
...,...,...,...
1578895,27511,3323773,5.0
1578896,27511,1490790,5.0
1578897,27511,2280040,5.0
1578898,27511,3298475,4.0


In [245]:
reviews_df.to_csv('reviews_cleaned.csv')
unique_ingred_df.to_csv('ingredient.csv')
recipes_df.to_csv('recipes_cleaned.csv')
ingredient_df_indexed.to_csv('ingredient_recipe.csv')

In [246]:
from google.colab import files
files.download('ingredient.csv')
files.download('recipes_cleaned.csv')
files.download('reviews_cleaned.csv')
files.download('ingredient_recipe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>